In [6]:
# simple_credit_risk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve


# Step 1. Create dummy dataset

np.random.seed(42)
n = 500  # number of customers

# Features
age = np.random.randint(21, 65, n)
income = np.random.normal(50000, 15000, n)  # average income 50k
loan_amount = np.random.normal(15000, 5000, n)
loan_to_income = loan_amount / (income + 1)

# Probability of default (synthetic rule: higher loan_to_income → higher risk)
prob_default = 1 / (1 + np.exp(-( -3 + 5 * loan_to_income + 0.02 * (65 - age) )))

# Target variable (default = 1, non-default = 0)
default = np.random.binomial(1, prob_default)

# Combine into DataFrame
df = pd.DataFrame({
    "age": age,
    "income": income,
    "loan_amount": loan_amount,
    "loan_to_income": loan_to_income,
    "default": default
})

print("Sample data:")
print(df.head())

# Step 2. Train/test split

X = df.drop(columns=["default"])
y = df["default"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Step 3. Train model

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


# Step 4. Predictions

y_pred = model.predict_proba(X_test)[:, 1]


# Step 5. Evaluation

auc = roc_auc_score(y_test, y_pred)
fpr, tpr, _ = roc_curve(y_test, y_pred)
ks = max(tpr - fpr)

print(f"AUC: {auc:.3f}")
print(f"KS : {ks:.3f}")


# Step 6. Save results

results = X_test.copy()
results["actual"] = y_test.values
results["score"] = y_pred
results.to_csv("scored_test.csv", index=False)
print("Scored test set saved to scored_test.csv")


Sample data:
   age        income   loan_amount  loan_to_income  default
0   59  79269.201286  21422.665170        0.270249        0
1   49  49398.722939  18445.181016        0.373386        1
2   35  57941.532675  19188.487192        0.331164        1
3   63  47241.623934  21839.926537        0.462293        0
4   28  48648.820502  21774.225862        0.447571        1
AUC: 0.699
KS : 0.320
Scored test set saved to scored_test.csv
